# 파일 불러오기

In [1]:
import glob
import os
import re
import numpy
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricst/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

print(txt_list[:10])


['/aiffel/aiffel/lyricst/data/lyrics/janisjoplin.txt', '/aiffel/aiffel/lyricst/data/lyrics/nursery_rhymes.txt', '/aiffel/aiffel/lyricst/data/lyrics/dickinson.txt', '/aiffel/aiffel/lyricst/data/lyrics/nicki-minaj.txt', '/aiffel/aiffel/lyricst/data/lyrics/disney.txt', '/aiffel/aiffel/lyricst/data/lyrics/bob-marley.txt', '/aiffel/aiffel/lyricst/data/lyrics/beatles.txt', '/aiffel/aiffel/lyricst/data/lyrics/dj-khaled.txt', '/aiffel/aiffel/lyricst/data/lyrics/kanye.txt', '/aiffel/aiffel/lyricst/data/lyrics/patti-smith.txt']


# 데이터 정제 과정

In [2]:
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Busted flat in Baton Rouge, waitin' for a train", "And I's feelin' near as faded as my jeans", 'Bobby thumbed a diesel down, just before it rained']


In [3]:
#문장을 전처리시키는 함수입니다.
def preprosentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    
    return sentence

corpus = []
for save in raw_corpus:
    x = preprosentence(save)
    corpus.append(x)

print(corpus[:3])

['<start> busted flat in baton rouge , waitin for a train <end>', '<start> and i s feelin near as faded as my jeans <end>', '<start> bobby thumbed a diesel down , just before it rained <end>']


In [4]:
def tokenize(corpus1):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words =12000, filters=' ', oov_token="<unk>")
    tokenizer.fit_on_texts(corpus1)
    tensor = tokenizer.texts_to_sequences(corpus1)
    
    
    tensor_fifteen = []
    for tensor_a in tensor:
        if len(tensor_a) >= 15:
            tensor_fifteen.append(tensor_a)
        else:
            continue
            
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor_fifteen,padding='post',maxlen=20)
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  11 2706  131 ...  456 6490    3]
 [   2 5633 7665 ...    0    0    0]
 [   2   23 1731 ...  383    3    0]
 ...
 [   2    5   22 ...    0    0    0]
 [   2    5   22 ...    0    0    0]
 [   2    5   22 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f68941f7df0>


# 학습을 위한 모델 구조 만들기

In [5]:
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:] 

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

In [6]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 128
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((128, 19), (128, 19)), types: (tf.int32, tf.int32)>

In [7]:
BUFFER_SIZE = len(enc_val)
BATCH_SIZE = 128
steps_per_epoch = len(enc_val) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset1 = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset1 = dataset1.shuffle(BUFFER_SIZE)
dataset1 = dataset1.batch(BATCH_SIZE, drop_remainder=True)
dataset1

<BatchDataset shapes: ((128, 19), (128, 19)), types: (tf.int32, tf.int32)>

In [8]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 1280
hidden_size = 1300
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) 

# 모델 학습 및 평가

In [9]:
optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다

#Loss
# tf.keras.losses.SparseCategoricalCrossentropy : https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none') # 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.

In [10]:
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
model.fit(dataset, epochs=10, validation_data=dataset1)

#Val_Loss값 최소 3.5479

Epoch 1/10
161/161 [==============================] - 44s 222ms/step - loss: 5.2741 - val_loss: 4.8326
Epoch 2/10
161/161 [==============================] - 38s 236ms/step - loss: 4.5402 - val_loss: 4.4244
Epoch 3/10
161/161 [==============================] - 41s 252ms/step - loss: 4.2306 - val_loss: 4.2410
Epoch 4/10
161/161 [==============================] - 41s 255ms/step - loss: 4.0330 - val_loss: 4.1193
Epoch 5/10
161/161 [==============================] - 41s 255ms/step - loss: 3.8633 - val_loss: 4.0164
Epoch 6/10
161/161 [==============================] - 41s 257ms/step - loss: 3.7104 - val_loss: 3.9351
Epoch 7/10
161/161 [==============================] - 41s 257ms/step - loss: 3.5684 - val_loss: 3.8732
Epoch 8/10
161/161 [==============================] - 41s 255ms/step - loss: 3.4284 - val_loss: 3.8151
Epoch 9/10
161/161 [==============================] - 41s 256ms/step - loss: 3.2931 - val_loss: 3.7712
Epoch 10/10
161/161 [==============================] - 41s 256ms/step - l

In [11]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis = -1), axis = -1)[:, -1]
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis = -1)
        
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len:break
            
        generated = ""
        for word_index in test_tensor[0].numpy():
            generated += tokenizer.index_word[word_index] + " "
        
        return generated

In [12]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you '

학습을 돌리는 과정에서 val_loss값이 2.2이하로 낮아지질 않아서 배치,임베딩,은닉사이즈를 늘려보고 줄여봐도
2.2이하로 내리진 못했습니다. 그래도 문장을 생성하는것에는 문제가 없습니다.